In [2]:
import torch

# בדיקה אם ה-GPU זמין
if torch.cuda.is_available():
    print("CUDA is available. GPU is detected.")
    # הגדרת המכשיר ל-GPU
    device = torch.device("cuda")
    # יצירת טנזור והעברה ל-GPU
    x = torch.tensor([1.0, 2.0, 3.0], device=device)
    y = torch.tensor([4.0, 5.0, 6.0], device=device)
    # חישוב על ה-GPU
    z = x + y
    print(f"Result of tensor addition on GPU: {z}")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is available. GPU is detected.
Result of tensor addition on GPU: tensor([5., 7., 9.], device='cuda:0')


In [11]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical


In [5]:
data_dir = "data\\asl_alphabet_train"
categories = os.listdir(data_dir)
img_size = 64

<h1>Pre-Processing the Data<h1>

in this section we will create a trainig data model.
<li>this model will present by array of pairs.
<li>each pair will be a pixels of image array and class of this image.
<li>for example : the letter 'A' will be [pixel array of image A , the class that present A (0)]

In [6]:
def create_training_data():
    training_data = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_array = cv2.resize(img_array, (img_size, img_size))
                training_data.append([resized_array, class_num])
            except Exception as e:
                pass
    return training_data

training_data = create_training_data()
print(f"Number of samples: {len(training_data)}")

Number of samples: 87000


now we make this steps : 
<li> we shuffle all the data from the traing_data
<li> we split the training data to x_train and y_train
<li> the x_train will contain the features for us it will be the pixel array of each image
<li> the y_train will contain the labels for us it will be the class that present each image (0,1,2...)
<li> than we normelize the x_train to recive values for each array in values between 0-1
<li> we transfer the y_train to categorical for the cnn model 

In [14]:
# now we shuffle and split the data
import random
random.shuffle(training_data)

x_train = []
y_train = []

for features,label in training_data: #split the training data to x_train that contain the image array , and y_train that contain the class name (index)
    x_train.append(features)
    y_train.append(label)

x_train = np.array(x_train).reshape(-1,img_size,img_size,1) / 255.0 # normalize the values in the x_train to numbers between 0-1 
y_train = to_categorical(y_train, num_classes=len(categories))


<h1>Building a CNN model

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential() #create the model 

#first convolutional layer:
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model.add(MaxPooling2D((2, 2)))

# Second convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten and dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(categories), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

c:\Users\amitx\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,289,629 (12.55 MB)

 Trainable params: 3,289,629 (12.55 MB)

 Non-trainable params: 0 (0.00 B)